In [1]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from bayes_opt import BayesianOptimization
from sklearn.metrics import roc_auc_score

import time, warnings, json, gc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb

In [2]:
def verbalise_dataset(train, test):
    print('Train shape:' + str(train.shape))
    print('Test shape:' + str(test.shape))
    print()

In [3]:
def load_file(filepath):

    start_time = time.time()
    df = pd.read_csv(filepath, low_memory=False)
    elapsed_time = time.time() - start_time
    print("Dataset loaded, time elapsed: " + str(elapsed_time))

    return df

train = load_file('../data/train.csv')  # (76020, 371)
test = load_file('../data/test.csv')  # (75818, 370)
verbalise_dataset(train, test)

Dataset loaded, time elapsed: 15.59230661392212
Dataset loaded, time elapsed: 17.333653926849365
Train shape:(76020, 371)
Test shape:(75818, 370)



In [4]:
def remove_duplicate_col(train, test):

    print('Removing duplicated features')
    output = []
    columns = train.columns  # list of headers
    for i in range(len(columns)-1):
        for j in range(i+1,len(columns)):
            if np.array_equal(train[columns[i]].values, train[columns[j]].values) and columns[j] not in output:
                    output.append(columns[j])
    
    train = train.drop(output, axis=1)
    test = test.drop(output, axis=1)

    return train, test


clean_train, clean_test = remove_duplicate_col(train, test)
verbalise_dataset(clean_train, clean_test)

Removing duplicated features
Train shape:(76020, 309)
Test shape:(75818, 308)



In [5]:
def remove_constant_col(train, test):

    print('Removing constant features')
    columns = []
    for col in train.columns:
        if train[col].std() == 0:
            columns.append(col)

    train = train.drop(columns, axis=1)
    test = test.drop(columns, axis=1)

    return train, test

clean_train, clean_test = remove_constant_col(clean_train, clean_test)
verbalise_dataset(clean_train, clean_test)

Removing constant features
Train shape:(76020, 308)
Test shape:(75818, 307)



In [6]:
# split data into train and test
X = clean_train.drop(["TARGET","ID"],axis=1)
Y = clean_train['TARGET'].values

test_id = clean_test.ID
test = clean_test.drop(["ID"],axis=1)
seeds = [2534324, 13454236, 34623, 1367457, 12321]

In [8]:
def lgbm_evaluate(**params):
    warnings.simplefilter('ignore')

    params['num_leaves'] = int(params['num_leaves'])
    params['max_depth'] = int(params['max_depth'])

    test_pred_proba = np.zeros(len(X))
    
    n_splits = 10
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=2534324)

    for train_idx, valid_idx in skf.split(X, Y):

        # Feature selection
        clf = lgb.LGBMClassifier()
        selector = clf.fit(X, Y)
        fs = SelectFromModel(selector, prefit=True)
        train_df = fs.transform(X)
        test_df = fs.transform(test)

        train_x, train_y = train_df[train_idx], Y[train_idx]
        valid_x, valid_y = train_df[valid_idx], Y[valid_idx]

        clf = lgb.LGBMClassifier(**params, n_estimators=100, nthread=4)
        clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric='auc', verbose=False, early_stopping_rounds=100)

        test_pred_proba[valid_idx] = clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:, 1]

        del train_x, train_y, valid_x, valid_y
        gc.collect()

    return roc_auc_score(Y, test_pred_proba)


if __name__ == "__main__":

    # set to ignore warnings
    warnings.simplefilter(action='ignore', category=FutureWarning)
    
    params = {
        'learning_rate': (.01, .02),
        'num_leaves': (20, 35),
        'colsample_bytree': (0.8, 1),
        'subsample': (0.8, 1),
        'max_depth': (7, 9),
        'reg_alpha': (.03, .05),
        'reg_lambda': (.06, .08),
        'min_split_gain': (.01, .03),
        'min_child_weight': (38, 60)
    }

    bo = BayesianOptimization(lgbm_evaluate, params)
    bo.maximize(init_points=5, n_iter=10)
    best_params = bo.max['params']
    
    best_params['num_leaves'] = int(best_params['num_leaves'])

    print(best_params)
    # output copied and stored as params.json
    
    import json
    json.dump(best_params, open('params0.json', 'w'))


|   iter    |  target   | colsam... | learni... | max_depth | min_ch... | min_sp... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------
|  1        |  0.8342   |  0.9345   |  0.01596  |  7.028    |  54.64    |  0.02701  |  21.35    |  0.03943  |  0.06914  |  0.8195   |
|  2        |  0.8345   |  0.8545   |  0.01172  |  7.93     |  48.57    |  0.02514  |  34.93    |  0.0327   |  0.07505  |  0.8822   |
|  3        |  0.8351   |  0.811    |  0.01541  |  8.35     |  57.07    |  0.02991  |  24.16    |  0.03105  |  0.07476  |  0.9812   |
|  4        |  0.8362   |  0.8126   |  0.0175   |  7.329    |  46.07    |  0.01735  |  27.27    |  0.03911  |  0.07887  |  0.9795   |
|  5        |  0.835    |  0.9233   |  0.01777  |  8.507    |  58.11    |  0.01013  |  30.39    |  0.04956  |  0.0659   |  0.9153   |
|  6        |  0.8345   |  0.8016   |  0.01413  |  7.044    | 

In [10]:
def lgbm_evaluate(**params):
    warnings.simplefilter('ignore')

    params['num_leaves'] = int(params['num_leaves'])
    params['max_depth'] = int(params['max_depth'])

    test_pred_proba = np.zeros(len(X))
    
    n_splits = 10
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=13454236)

    for train_idx, valid_idx in skf.split(X, Y):

        # Feature selection
        clf = lgb.LGBMClassifier()
        selector = clf.fit(X, Y)
        fs = SelectFromModel(selector, prefit=True)
        train_df = fs.transform(X)
        test_df = fs.transform(test)

        train_x, train_y = train_df[train_idx], Y[train_idx]
        valid_x, valid_y = train_df[valid_idx], Y[valid_idx]

        clf = lgb.LGBMClassifier(**params, n_estimators=100, nthread=4)
        clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric='auc', verbose=False, early_stopping_rounds=100)

        test_pred_proba[valid_idx] = clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:, 1]

        del train_x, train_y, valid_x, valid_y
        gc.collect()

    return roc_auc_score(Y, test_pred_proba)


if __name__ == "__main__":

    # set to ignore warnings
    warnings.simplefilter(action='ignore', category=FutureWarning)
    
    params = {
        'learning_rate': (.01, .02),
        'num_leaves': (20, 35),
        'colsample_bytree': (0.8, 1),
        'subsample': (0.8, 1),
        'max_depth': (7, 9),
        'reg_alpha': (.03, .05),
        'reg_lambda': (.06, .08),
        'min_split_gain': (.01, .03),
        'min_child_weight': (38, 60)
    }

    bo = BayesianOptimization(lgbm_evaluate, params)
    bo.maximize(init_points=5, n_iter=10)
    best_params = bo.max['params']
    
    best_params['num_leaves'] = int(best_params['num_leaves'])

    print(best_params)
    # output copied and stored as params.json
    
    import json
    json.dump(best_params, open('params1.json', 'w'))


|   iter    |  target   | colsam... | learni... | max_depth | min_ch... | min_sp... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------
|  1        |  0.8357   |  0.9047   |  0.01831  |  7.939    |  50.29    |  0.01127  |  29.09    |  0.04976  |  0.07681  |  0.8951   |
|  2        |  0.8355   |  0.804    |  0.01725  |  8.783    |  51.81    |  0.0104   |  26.1     |  0.04386  |  0.06591  |  0.9893   |
|  3        |  0.832    |  0.9728   |  0.01259  |  7.008    |  49.28    |  0.02042  |  22.09    |  0.03606  |  0.07453  |  0.8347   |
|  4        |  0.8353   |  0.8257   |  0.01588  |  7.097    |  54.85    |  0.01727  |  27.24    |  0.0425   |  0.07836  |  0.9037   |
|  5        |  0.8338   |  0.9153   |  0.01455  |  7.71     |  55.07    |  0.0142   |  24.44    |  0.03822  |  0.06851  |  0.9114   |
|  6        |  0.8355   |  0.9579   |  0.0168   |  8.873    | 

In [11]:
def lgbm_evaluate(**params):
    warnings.simplefilter('ignore')

    params['num_leaves'] = int(params['num_leaves'])
    params['max_depth'] = int(params['max_depth'])

    test_pred_proba = np.zeros(len(X))
    
    n_splits = 10
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=34623)

    for train_idx, valid_idx in skf.split(X, Y):

        # Feature selection
        clf = lgb.LGBMClassifier()
        selector = clf.fit(X, Y)
        fs = SelectFromModel(selector, prefit=True)
        train_df = fs.transform(X)
        test_df = fs.transform(test)

        train_x, train_y = train_df[train_idx], Y[train_idx]
        valid_x, valid_y = train_df[valid_idx], Y[valid_idx]

        clf = lgb.LGBMClassifier(**params, n_estimators=100, nthread=4)
        clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric='auc', verbose=False, early_stopping_rounds=100)

        test_pred_proba[valid_idx] = clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:, 1]

        del train_x, train_y, valid_x, valid_y
        gc.collect()

    return roc_auc_score(Y, test_pred_proba)


if __name__ == "__main__":

    # set to ignore warnings
    warnings.simplefilter(action='ignore', category=FutureWarning)
    
    params = {
        'learning_rate': (.01, .02),
        'num_leaves': (20, 35),
        'colsample_bytree': (0.8, 1),
        'subsample': (0.8, 1),
        'max_depth': (7, 9),
        'reg_alpha': (.03, .05),
        'reg_lambda': (.06, .08),
        'min_split_gain': (.01, .03),
        'min_child_weight': (38, 60)
    }

    bo = BayesianOptimization(lgbm_evaluate, params)
    bo.maximize(init_points=5, n_iter=10)
    best_params = bo.max['params']
    
    best_params['num_leaves'] = int(best_params['num_leaves'])

    print(best_params)
    # output copied and stored as params.json
    
    import json
    json.dump(best_params, open('params2.json', 'w'))


|   iter    |  target   | colsam... | learni... | max_depth | min_ch... | min_sp... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------
|  1        |  0.8347   |  0.8414   |  0.01835  |  7.551    |  59.61    |  0.01476  |  32.0     |  0.03484  |  0.06161  |  0.8317   |
|  2        |  0.8346   |  0.8574   |  0.01535  |  7.52     |  39.4     |  0.02904  |  25.05    |  0.04591  |  0.06508  |  0.9961   |
|  3        |  0.8344   |  0.9869   |  0.0183   |  8.44     |  39.95    |  0.01208  |  23.59    |  0.032    |  0.07836  |  0.9315   |
|  4        |  0.8345   |  0.9408   |  0.01598  |  7.669    |  47.79    |  0.01836  |  27.55    |  0.04255  |  0.07719  |  0.9376   |
|  5        |  0.8348   |  0.8074   |  0.01762  |  8.783    |  53.28    |  0.02314  |  22.15    |  0.04489  |  0.07063  |  0.8863   |
|  6        |  0.8342   |  0.8819   |  0.01543  |  7.11     | 

In [12]:
def lgbm_evaluate(**params):
    warnings.simplefilter('ignore')

    params['num_leaves'] = int(params['num_leaves'])
    params['max_depth'] = int(params['max_depth'])

    test_pred_proba = np.zeros(len(X))
    
    n_splits = 10
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=1367457)

    for train_idx, valid_idx in skf.split(X, Y):

        # Feature selection
        clf = lgb.LGBMClassifier()
        selector = clf.fit(X, Y)
        fs = SelectFromModel(selector, prefit=True)
        train_df = fs.transform(X)
        test_df = fs.transform(test)

        train_x, train_y = train_df[train_idx], Y[train_idx]
        valid_x, valid_y = train_df[valid_idx], Y[valid_idx]

        clf = lgb.LGBMClassifier(**params, n_estimators=100, nthread=4)
        clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric='auc', verbose=False, early_stopping_rounds=100)

        test_pred_proba[valid_idx] = clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:, 1]

        del train_x, train_y, valid_x, valid_y
        gc.collect()

    return roc_auc_score(Y, test_pred_proba)


if __name__ == "__main__":

    # set to ignore warnings
    warnings.simplefilter(action='ignore', category=FutureWarning)
    
    params = {
        'learning_rate': (.01, .02),
        'num_leaves': (20, 35),
        'colsample_bytree': (0.8, 1),
        'subsample': (0.8, 1),
        'max_depth': (7, 9),
        'reg_alpha': (.03, .05),
        'reg_lambda': (.06, .08),
        'min_split_gain': (.01, .03),
        'min_child_weight': (38, 60)
    }

    bo = BayesianOptimization(lgbm_evaluate, params)
    bo.maximize(init_points=5, n_iter=10)
    best_params = bo.max['params']
    
    best_params['num_leaves'] = int(best_params['num_leaves'])

    print(best_params)
    # output copied and stored as params.json
    
    import json
    json.dump(best_params, open('params3.json', 'w'))


|   iter    |  target   | colsam... | learni... | max_depth | min_ch... | min_sp... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------
|  1        |  0.8361   |  0.8415   |  0.01971  |  8.745    |  54.06    |  0.01049  |  26.36    |  0.04994  |  0.06903  |  0.9926   |
|  2        |  0.8328   |  0.9919   |  0.01151  |  8.218    |  58.65    |  0.01025  |  29.8     |  0.04028  |  0.07583  |  0.8913   |
|  3        |  0.8348   |  0.9292   |  0.01589  |  8.94     |  48.14    |  0.01227  |  25.5     |  0.0481   |  0.07888  |  0.9536   |
|  4        |  0.8354   |  0.9435   |  0.0184   |  7.655    |  53.89    |  0.02835  |  27.56    |  0.04113  |  0.07838  |  0.9748   |
|  5        |  0.8339   |  0.8557   |  0.01007  |  8.565    |  54.05    |  0.01148  |  28.9     |  0.04774  |  0.06122  |  0.9773   |
|  6        |  0.833    |  0.8808   |  0.01197  |  7.658    | 

In [13]:
def lgbm_evaluate(**params):
    warnings.simplefilter('ignore')

    params['num_leaves'] = int(params['num_leaves'])
    params['max_depth'] = int(params['max_depth'])

    test_pred_proba = np.zeros(len(X))
    
    n_splits = 10
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=12321)

    for train_idx, valid_idx in skf.split(X, Y):

        # Feature selection
        clf = lgb.LGBMClassifier()
        selector = clf.fit(X, Y)
        fs = SelectFromModel(selector, prefit=True)
        train_df = fs.transform(X)
        test_df = fs.transform(test)

        train_x, train_y = train_df[train_idx], Y[train_idx]
        valid_x, valid_y = train_df[valid_idx], Y[valid_idx]

        clf = lgb.LGBMClassifier(**params, n_estimators=100, nthread=4)
        clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric='auc', verbose=False, early_stopping_rounds=100)

        test_pred_proba[valid_idx] = clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:, 1]

        del train_x, train_y, valid_x, valid_y
        gc.collect()

    return roc_auc_score(Y, test_pred_proba)


if __name__ == "__main__":

    # set to ignore warnings
    warnings.simplefilter(action='ignore', category=FutureWarning)
    
    params = {
        'learning_rate': (.01, .02),
        'num_leaves': (20, 35),
        'colsample_bytree': (0.8, 1),
        'subsample': (0.8, 1),
        'max_depth': (7, 9),
        'reg_alpha': (.03, .05),
        'reg_lambda': (.06, .08),
        'min_split_gain': (.01, .03),
        'min_child_weight': (38, 60)
    }

    bo = BayesianOptimization(lgbm_evaluate, params)
    bo.maximize(init_points=5, n_iter=10)
    best_params = bo.max['params']
    
    best_params['num_leaves'] = int(best_params['num_leaves'])

    print(best_params)
    # output copied and stored as params.json
    
    import json
    json.dump(best_params, open('params4.json', 'w'))

|   iter    |  target   | colsam... | learni... | max_depth | min_ch... | min_sp... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------
|  1        |  0.834    |  0.8049   |  0.01419  |  8.289    |  57.73    |  0.0221   |  23.64    |  0.03556  |  0.06523  |  0.8815   |
|  2        |  0.8335   |  0.9196   |  0.01121  |  7.926    |  41.79    |  0.01644  |  30.37    |  0.04919  |  0.07983  |  0.926    |
|  3        |  0.8353   |  0.8188   |  0.01494  |  7.234    |  48.02    |  0.02922  |  30.8     |  0.03877  |  0.07062  |  0.8606   |
|  4        |  0.8341   |  0.8876   |  0.0158   |  8.743    |  56.29    |  0.01104  |  25.55    |  0.0494   |  0.06515  |  0.8318   |
|  5        |  0.8329   |  0.9648   |  0.01249  |  7.564    |  39.27    |  0.01019  |  24.17    |  0.04755  |  0.068    |  0.9083   |
|  6        |  0.8341   |  0.9404   |  0.01836  |  7.001    | 

In [ ]:
# https://towardsdatascience.com/automated-machine-learning-hyperparameter-tuning-in-python-dfda59b72f8a